# Example usage

## Version check

In [1]:
import bumbag

print(bumbag.__version__)

4.0.0


### `daycount`

Task: The digits of 22 February 2022 form [a palindrome and an ambigram](https://en.wikipedia.org/wiki/Twosday) in dd-mm-yyyy format.
List the next five dates with these properties.

In [2]:
from datetime import date

from toolz import curried

import bumbag


def format_date(date, with_dash):
    fmt = "%d-%m-%Y" if with_dash else "%d%m%Y"
    return date.strftime(fmt)


def is_palindrome_date(date):
    d = format_date(date, False)
    return d == d[::-1]


def is_ambigram_date(date):
    d = format_date(date, False)
    return set(d) <= {"0", "1", "2", "8"}


def show_date(date):
    return f"{format_date(date, True)} ↦ {format_date(date, False)}"


curried.pipe(
    bumbag.daycount(date(2022, 2, 23)),
    curried.filter(is_palindrome_date),
    curried.filter(is_ambigram_date),
    curried.map(show_date),
    curried.take(5),
    list,
)

['08-02-2080 ↦ 08022080',
 '18-02-2081 ↦ 18022081',
 '28-02-2082 ↦ 28022082',
 '10-12-2101 ↦ 10122101',
 '20-12-2102 ↦ 20122102']

### `flatten`

In [3]:
import bumbag

irregular_list = [
    ["one", 2],
    3,
    [(4, "five")],
    [[["six"]]],
    "seven",
    [],
]

list(bumbag.flatten(irregular_list, 8, [9, ("ten",)]))

['one', 2, 3, 4, 'five', 'six', 'seven', 8, 9, 'ten']

### `humantime`

Use as decorator:

In [4]:
import time
import functools

import bumbag


def stopwatch(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        t0 = time.time()
        output = func(*args, **kwargs)
        runtime = time.time() - t0
        print(f"{func.__name__} took {bumbag.humantime(runtime)}")
        return output

    return wrapper


@stopwatch
def my_function():
    time.sleep(0.1)

In [5]:
my_function()

my_function took 0.100308s


Use as context manager:

In [6]:
import time
from contextlib import contextmanager

import bumbag


@contextmanager
def stopwatch():
    t0 = time.time()
    yield
    runtime = time.time() - t0
    print(f"{bumbag.humantime(runtime)}")

In [7]:
with stopwatch():
    time.sleep(0.1)

0.100497s


### `two_set_summary`

In [8]:
import bumbag

x = {"a", "c", "b", "g", "h"}
y = {"c", "d", "e", "f", "g"}
summary = bumbag.two_set_summary(x, y)
print(summary["report"])

    x (n=5): {'a', 'b', 'c', ...}
    y (n=5): {'c', 'd', 'e', ...}
x | y (n=8): {'a', 'b', 'c', ...}
x & y (n=2): {'c', 'g'}
x - y (n=3): {'a', 'b', 'h'}
y - x (n=3): {'d', 'e', 'f'}
x ^ y (n=6): {'a', 'b', 'd', ...}
jaccard = 0.25
overlap = 0.4
dice = 0.4
disjoint?: False
x == y: False
x <= y: False
x <  y: False
y <= x: False
y <  x: False
